In [7]:
import numpy as np
import tensorflow as tf

import model
import anchors
import generate_data_sequence

In [8]:
GROUND_TRUTH_BBOX_PATH = "../bbox_galbe_2.csv"
DATA_PATH = "/home/cogrannr/roues/MEFRO/grises/img_galbe_avec_defauts/"

base_model_name = "base_model_galbe.h5"

IMAGE_SHAPE = (2792,99)  # Images resize en taille ~ moyenne : moyenne pondérée
INPUT_IMAGE_SHAPE = (99,2792,1) # shape qui rentre dans le cnn
# même taille pour toutes les parties d'images, présentes en même densité

DATA_SIZE = int(5000) # taille des données (test/valid/test compris), max : 73K


RATIO_X = 4
RATIO_Y = 8

BATCH_SIZE = 1
ANCHOR_BATCH_SIZE = 8  # 4 pos et 4 neg
ratio_batch = 0.5

anchor_sizes = ((10,20),
                (20,40),
                (35,70),
                (50,100),
                (10,10),
                (15,15),
                (20,20),
                (25,25),
                (30,30),
                (40,40),
                (50,50))


LEARNING_RATE = 0.0001
NUM_EPOCHS = 50

In [9]:

anchors_array = anchors.generate(INPUT_IMAGE_SHAPE,RATIO_X,RATIO_Y,anchor_sizes)
print(anchors_array.shape)


train_dataset = generate_data_sequence.Dataset_sequence("train",DATA_SIZE, IMAGE_SHAPE,ANCHOR_BATCH_SIZE,
    ratio_batch, DATA_PATH, GROUND_TRUTH_BBOX_PATH, anchors_array)

valid_dataset = generate_data_sequence.Dataset_sequence("valid",DATA_SIZE, IMAGE_SHAPE,ANCHOR_BATCH_SIZE,
    ratio_batch, DATA_PATH, GROUND_TRUTH_BBOX_PATH, anchors_array)

test_dataset  = generate_data_sequence.Dataset_sequence("test",DATA_SIZE, IMAGE_SHAPE,ANCHOR_BATCH_SIZE,
    ratio_batch, DATA_PATH, GROUND_TRUTH_BBOX_PATH, anchors_array)


model_creator = model.Model_creator(INPUT_IMAGE_SHAPE,anchors_array.shape,
                                    LEARNING_RATE,base_model_name,BATCH_SIZE)
rpn = model_creator.init_RPN_model()

rpn.summary()


(8376, 11, 4)


TypeError: parametrize_prediction() got an unexpected keyword argument 'target_anchors'

In [4]:


"""call_backs_list = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath ='rpn.h5', monitor ="val_accuracy", save_best_only = True)]
"""
#callbacks=call_backs_list


"""mirrored_strategy = tf.distribute.MirroredStrategy()

with mirrored_strategy.scope():
"""
rpn.fit(train_dataset, epochs=NUM_EPOCHS, validation_data=valid_dataset,
    max_queue_size=50, workers=10, use_multiprocessing=False)

# use_multiprocessing  : pas obligé de le mettre e True, peut etre freeze, a voir



Train for 3000 steps
Epoch 1/50
3000/3000 [==============================] - 42s 14ms/step - loss: 1.6205 - cls_pred_loss: 0.7125 - reg_pred_loss: 0.9080 - cls_pred_accuracy: 0.0407 - cls_pred_precision: 8.8171e-05
Epoch 2/50
3000/3000 [==============================] - 40s 13ms/step - loss: 1.3422 - cls_pred_loss: 0.7025 - reg_pred_loss: 0.6397 - cls_pred_accuracy: 0.0195 - cls_pred_precision: 8.7421e-05
Epoch 3/50
3000/3000 [==============================] - 39s 13ms/step - loss: 1.2417 - cls_pred_loss: 0.6959 - reg_pred_loss: 0.5458 - cls_pred_accuracy: 0.0080 - cls_pred_precision: 8.6378e-05
Epoch 4/50
3000/3000 [==============================] - 39s 13ms/step - loss: 1.1803 - cls_pred_loss: 0.6960 - reg_pred_loss: 0.4843 - cls_pred_accuracy: 0.0053 - cls_pred_precision: 8.7168e-05
Epoch 5/50
3000/3000 [==============================] - 39s 13ms/step - loss: 1.1416 - cls_pred_loss: 0.6934 - reg_pred_loss: 0.4483 - cls_pred_accuracy: 0.0033 - cls_pred_precision: 8.5948e-05
Epoch 6/5

KeyboardInterrupt: 

In [12]:
for (x,y) in train_dataset:
    break
reg = y[1]
print(reg[reg!=0])

rpn(x)

tf.Tensor(
[-0.03        0.28       -0.05551732 -0.14874165  0.01        0.12
 -0.05551732 -0.14874165 -0.03       -0.04       -0.05551732 -0.14874165
  0.09       -0.04       -0.05551732 -0.14874165], shape=(16,), dtype=float64)


[<tf.Tensor: shape=(1, 8376, 11), dtype=float32, numpy=
 array([[[0.44460377, 0.06790411, 0.02122025, ..., 0.8999921 ,
          0.9632686 , 0.9086206 ],
         [0.78575116, 0.778725  , 0.5077777 , ..., 0.00974547,
          0.13480666, 0.29174733],
         [0.1907199 , 0.5662457 , 0.69801486, ..., 0.01462257,
          0.19660813, 0.71539724],
         ...,
         [0.4502523 , 0.56781596, 0.34713036, ..., 0.56947255,
          0.48298433, 0.7322713 ],
         [0.4003859 , 0.4493301 , 0.28889224, ..., 0.37677762,
          0.4620861 , 0.5385167 ],
         [0.48268422, 0.50383997, 0.41157487, ..., 0.39422536,
          0.61972314, 0.6019927 ]]], dtype=float32)>,
 <tf.Tensor: shape=(1, 8376, 44), dtype=float32, numpy=
 array([[[ 0.01922669,  2.1157696 ,  2.7192845 , ..., -0.01384572,
          -1.289913  , -1.522272  ],
         [-0.6915438 ,  1.2968987 , -1.0298516 , ...,  1.4164687 ,
           1.6289655 , -2.5401742 ],
         [ 0.7464298 ,  0.68734145, -0.03857441, ...,  0.97

In [ ]:

mirrored_strategy = tf.distribute.MirroredStrategy()

with mirrored_strategy.scope():
    
    anchors_array = anchors.generate(INPUT_IMAGE_SHAPE,RATIO_X,RATIO_Y,anchor_sizes)
    print(anchors_array.shape)


    model_creator = model.Model_creator(INPUT_IMAGE_SHAPE,anchors_array.shape,
                                    LEARNING_RATE,base_model_name,BATCH_SIZE)
    rpn = model_creator.init_RPN_model()


    train_dataset = generate_data_sequence.Dataset_sequence("train",DATA_SIZE, IMAGE_SHAPE,ANCHOR_BATCH_SIZE,
        ratio_batch, DATA_PATH, GROUND_TRUTH_BBOX_PATH, anchors_array)

    valid_dataset = generate_data_sequence.Dataset_sequence("valid",DATA_SIZE, IMAGE_SHAPE,ANCHOR_BATCH_SIZE,
        ratio_batch, DATA_PATH, GROUND_TRUTH_BBOX_PATH, anchors_array)

    
    rpn.fit(train_dataset, epochs=NUM_EPOCHS, validation_data=valid_dataset,
        max_queue_size=10, workers=1, use_multiprocessing=False)

